In [32]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import json
import time
import xlwt

In [33]:
qianxi = "https://qianxi.baidu.com/"
page = requests.get(qianxi)
soup = BeautifulSoup(page.text, 'lxml')
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<title>百度迁徙-百度地图慧眼</title>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="百度迁徙 人口迁徙 春运迁徙 迁徙 城市迁徙 春节迁徙" name="keywords"/>
<meta content="百度迁徙，通过将定位可视化直观呈现国内春运期间人口迁徙情况。" name="description"/>
<script>
    var _hmt = _hmt || [];
    (function() {
      var hm = document.createElement("script");
      hm.src = "https://hm.baidu.com/hm.js?f0abc658757a9f6131491a24a4112ebc";
      var s = document.getElementsByTagName("script")[0];
      s.parentNode.insertBefore(hm, s);
    })();
    </script>
<link href="//qianxi.cdn.bcebos.com/static/css/app/index.0f239095bfa1df57a1c4.css?6ecddb521a0373bf222f" rel="stylesheet"/></head>
<body>
<div id="content"></div>
<script src="//api.map.baidu.com/api?v=3.0&amp;ak=5ssIAkexwFSGMatjOF95gg3sjet3yxQ1&amp;callback=initMap" type="text/javascript"></script>
<!--<script src="https://code.bdstatic.com/npm/mapv@2.0.50

In [34]:
def CityName2Code(dt,cityName):

    """

    Arguments:

        dt {str} -- [description]

        cityName {str} -- [description]

    """    

    cityCode=''

    searchKey=''

    codeKey=''

    cityCodePath ='CityCode.json'

    #use utf-8-sig to avoid error

    data = open((cityCodePath), encoding='utf-8-sig')

    result = json.load(data)

    if dt=='province':

        searchKey='省名称'

        codeKey='省代码'

    elif dt =='city':

        searchKey='地级市名称'

        codeKey='地级市代码'

    for rowNum in range(len(result)):

        if result[rowNum][searchKey]==cityName:

            cityCode = result[rowNum][codeKey]

    return cityCode

def UrlFormat(rankMethod,dt,name,migrationType,date):

    """

    Arguments:

        rankMethod {str} -- city||province 获得数据的行政级别

        dt {str} -- city||province 中心地行政级别

        name {str} -- example:'温州市||浙江省' 作为中心地的地名

        migrationType {str} -- in||out

        date {str} -- example:20200202

    """    

    list_date = list(date)

    list_date.insert(4,'-')    

    list_date.insert(7,'-')

    formatDate = ''.join(list_date)

    formatDate= formatDate+" 00:00:00"

    timeArray = time.strptime(formatDate, "%Y-%m-%d %H:%M:%S")

    timeUnix = time.mktime(timeArray)

    ID = CityName2Code(dt,name)
    print(ID)

    url='http://huiyan.baidu.com/migration/{0}rank.jsonp?dt={1}&id={2}&type=move_{3}&date={4}&callback=jsonp_{5}000_0000000'.format(rankMethod,dt,ID,migrationType,date,int(timeUnix))

    print(url)
    
    return url

def JsonTextConvert(text):

    """Text2Json

    Arguments:

        text {str} -- webContent

    Returns:

        str -- jsonText

    """    

    text = text.encode('utf-8').decode('unicode_escape')

    head, sep, tail = text.partition('(')

    tail=tail.replace(")","")

    return tail

def GetData(rankMethod,dt,name,migrationType,date,isExcel):

    """

    Arguments:

        rankMethod {str} -- city||province 获得数据的行政级别

        dt {str} -- city||province 中心地行政级别

        name {str} -- example:'温州市||浙江省' 作为中心地的地名

        migrationType {str} -- in||out

        date {str} -- example:20200202

        isExcel {bool} -- true转出为excel格式

    """    

    r = requests.get(url=UrlFormat(rankMethod,dt,name,migrationType,date))

    text = r.text
    print(text)

    rawData=json.loads(JsonTextConvert(text))

    data= rawData['data']

    list = data['list']

    nameKey = ''

    if rankMethod=='province':

        nameKey = 'province_name'

    else:

        nameKey = 'city_name'

    if isExcel == True:

        workbook = xlwt.Workbook(encoding='utf-8') 

        worksheet = workbook.add_sheet('sheet1',cell_overwrite_ok=True) 

        table_head = [nameKey,'value']

        index = 1 

        for i in range(len(table_head)):

            worksheet.write(0,i,table_head[i])

        for l in list:

            worksheet.write(index,0,l[nameKey])

            worksheet.write(index,1,l['value'])

            index=index+1

        filename = name+date+'.xls'

        workbook.save('data/'+filename)

    else:

        for l in list:

            print(l[nameKey],':',l['value'])

In [35]:
GetData('province','province','湖北省','out','20200220',True)

420000
http://huiyan.baidu.com/migration/provincerank.jsonp?dt=province&id=420000&type=move_out&date=20200220&callback=jsonp_1582185600000_0000000
jsonp_1582185600000_0000000({"errno":0,"errmsg":"SUCCESS","data":{"list":[{"province_name":"\u6e56\u5357\u7701","value":24.52},{"province_name":"\u5e7f\u4e1c\u7701","value":19.92},{"province_name":"\u6cb3\u5357\u7701","value":13.58},{"province_name":"\u91cd\u5e86\u5e02","value":10.33},{"province_name":"\u6c5f\u897f\u7701","value":5.51},{"province_name":"\u6d59\u6c5f\u7701","value":4.5},{"province_name":"\u5b89\u5fbd\u7701","value":3.63},{"province_name":"\u9655\u897f\u7701","value":3.51},{"province_name":"\u6c5f\u82cf\u7701","value":2.7},{"province_name":"\u4e0a\u6d77\u5e02","value":2.25},{"province_name":"\u798f\u5efa\u7701","value":1.26},{"province_name":"\u56db\u5ddd\u7701","value":1.15},{"province_name":"\u5e7f\u897f\u58ee\u65cf\u81ea\u6cbb\u533a","value":1.04},{"province_name":"\u5c71\u4e1c\u7701","value":0.96},{"province_name":"\u8d35\